In [1]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

Cloning into 'newsjam'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 401 (delta 214), reused 245 (delta 108), pack-reused 0
Receiving objects: 100% (401/401), 9.34 MiB | 4.67 MiB/s, done.
Resolving deltas: 100% (214/214), done.


Imports (Run Once)

In [2]:
!pip install selenium --upgrade
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install tweepy --upgrade
!pip install python-dotenv --upgrade

     |████████████████████████████████| 958 kB 4.7 MB/s 
     |████████████████████████████████| 138 kB 75.9 MB/s 
     |████████████████████████████████| 356 kB 71.8 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 3.6 MB 50.9 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following addi

In [3]:
!python -m pip install -r requirements.txt
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 60 kB 3.2 MB/s 
     |████████████████████████████████| 306 kB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 61.1 MB/s 
     |████████████████████████████████| 3.4 MB 45.0 MB/s 
     |████████████████████████████████| 61 kB 473 kB/s 
     |████████████████████████████████| 3.3 MB 47.0 MB/s 
     |████████████████████████████████| 596 kB 31.3 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 243 kB 62.5 MB/s 
     |████████████████████████████████| 1.1 MB 61.7 MB/s 
     |████████████████████████████████| 132 kB 62.0 MB/s 
     |████████████████████████████████| 192 kB 58.2 MB/s 
     |████████████████████████████████| 271 kB 63.6 MB/s 
     |████████████████████████████████| 160 kB 60.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 14.

Twitter Setup and Authentication

In [2]:
import os
from dotenv import load_dotenv
from summ.lsa import LSASummarizer
from data import scraper_functions
import tweepy

#this will not work if you don't have the .env file with the API keys

load_dotenv('/content/API.env')
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
bearer = os.getenv('bearer')
access_key = os.getenv('access_key')
access_key_secret = os.getenv('access_key_secret')

client = tweepy.Client(bearer_token=bearer, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_key, access_token_secret=access_key_secret)

In [3]:
'''
temporary code while classifier is not implemented - just replace the URL with whatever one you want
'''

text, url = scraper_functions.actu_scraper('https://actu.fr/pays-de-la-loire/chateau-gontier-sur-mayenne_53062/sud-mayenne-face-a-la-5e-vague-de-covid-19-de-nouvelles-restrictions-a-l-hopital-et-dans-les-ehpad_47699637.html')

In [5]:
'''
temporary code while the classifier is not implemented
'''

lsa_summ = LSASummarizer()
summary = lsa_summ.get_summary(text)
summary = summary[0]
tweet_response = client.create_tweet(text=summary)
client.create_tweet(text=url, in_reply_to_tweet_id=tweet_response.data['id']) #reply to the posted tweet immediately with the original URL

Response(data={'id': '1478780224414355456', 'text': 'https://t.co/5gK5shenZC'}, includes={}, errors=[], meta={})

In [ ]:
'''
once we have the classifier, use this instead
runs the auto-scraper which grabs the URLs of several articles, classifies them until it receives a French one, then tweets it
'''

url_list = actu_autoscraper('https://actu.fr/le-journal-de-vitre/societe/coronavirus', url_amount=3)
lsa_summ = LSASummarizer()

for x in range(len(url_list)):
  text, url = actu_scraper(url_list[x])
  if text_classifier(text) == 0:
    continue
  else:
    summary = lsa_summ.get_summary(text)
    summary = summary[0]
    tweet_response = client.create_tweet(text=summary)
    client.create_tweet(text=url, in_reply_to_tweet_id=tweet_response.data['id']) #reply to the posted tweet immediately with the original URL
    break
  print("no suitable articles right now")